## Chain-of-Thought with no supporting context experiments

In [1]:
import sys, os
sys.path.append('..')
root = '../root/'

In [16]:
from util import summarize_trial, log_trial, save_agents
import joblib
from agents_ar import CoTAgent, ReflexionStrategy

In [6]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='D:/files_new/10-samples.json', split='train')

In [7]:
row = train_dataset[4]

In [17]:
row['question'] = 'تحتوي الحديقة على عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. كم عدد الزهور الحمراء الموجودة في الحديقة؟'
row['answer'] = 'بحسب المعلومات المقدمة، لا توجد زهور حمراء مذكورة في الحديقة.'

In [7]:
row

{'answer': 'بحسب المعلومات المقدمة، لا توجد زهور حمراء مذكورة في الحديقة.',
 'question': 'تحتوي الحديقة على عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. كم عدد الزهور الحمراء الموجودة في الحديقة؟'}

Refliction Strategy

In [8]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [18]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

In [19]:
from prompts_ar import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots_ar import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

In [25]:
agent = CoTAgent(question=row['question'],
                   context='',
                   key=row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples=COTQA_SIMPLE6,
                   reflect_prompt=cot_simple_reflect_prompt,
                   reflect_examples=COT_SIMPLE_REFLECTION)

In [9]:
print(row['question'])

تحتوي الحديقة على عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. كم عدد الزهور الحمراء الموجودة في الحديقة؟


In [21]:
output = agent.run(reflexion_strategy = strategy)

تفكير : دعونا نفكر خطوة بخطوة، لدينا عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. الأقحوان هي الزهور الحمراء، لذلك عدد الزهور الحمراء هو عدد الأقحوان الذي هو عشرة.
الرد: إنهاء[عشرة زهور أقحوان]


In [22]:
print(output)

None


In [26]:
from io import StringIO

# Create an in-memory buffer
captured_output = StringIO()

# Redirect the standard output temporarily to the buffer
original_stdout = sys.stdout
sys.stdout = captured_output

# Run the agent
agent.run(reflexion_strategy=strategy)

# Restore the original standard output
sys.stdout = original_stdout

# Get the captured conversation as a string
conversation_log = captured_output.getvalue().strip()

# Print the captured conversation (optional)
print(conversation_log)

تفكير : دعونا نفكر خطوة بخطوة، لدينا عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. الأقحوان هي الزهور الحمراء، لذلك عدد الزهور الحمراء في الحديقة هو عدد الأقحوان الذي هو عشرة.
الرد: إنهاء[عشرة زهور أقحوان]


In [27]:
print(conversation_log)

تفكير : دعونا نفكر خطوة بخطوة، لدينا عشرة زهور أقحوان واثنين من التوليب وثلاث من الخزامى. الأقحوان هي الزهور الحمراء، لذلك عدد الزهور الحمراء في الحديقة هو عدد الأقحوان الذي هو عشرة.
الرد: إنهاء[عشرة زهور أقحوان]


In [28]:
print(agent._build_agent_prompt())
print(agent._build_reflection_prompt())

نفذ التعليمات بدقة بناءً على تفكير منطقي للسياق. تأكد من أن ردك مُتناسق ومنطقي، مع استخدام خيار "إنهاء [الإجابة]" لإنهاء المهمة وإرسال ردك النهائي.
يمكنك مراجعة بعض الأمثلة التالية:

التعليمات: اكتب سلسلة من كلمات كل كلمة تبدأ بآخر حرف من الكلمة التي سبقتها، بدءًا بحرف "ب".
تفكير: دعونا نفكر خطوة بخطوة، يمكنني استخدام قائمة الكلمات العربية او القاموس لمعرفة الكلمات التي تبدأ بحرف "ب"، وان لا تزيد عن 8 كلمات. بعد اختيار كلمة تبدأ بحرف "ب"، احتاج إلى العثور على كلمة تبدأ بالحرف الأخير من تلك الكلمة، لذلك يمكنني كتابة "بقر، رمح، حليب، بحر، رحل".
الرد: إنهاء[سلسلة الكلمات: بقر، رمح، حليب، بحر، رحل]

التعليمات: إذا كان لدي أربعة صناديق، في كل صندوق ستة علب، كل علبة تحتوي على ثلاثة أقلام، فكم عدد الأقلام الكلي.
تفكير: دعونا نفكر خطوة بخطوة، في البداية لديك اربع صناديق بداخلها 6 علب، احتاج ان احسب العدد الكلي للعلب 4*6 =24  ، عدد العلب الكلي هو 24، كل علبة تحتوي على ثلاث أقلام، احتاج ان احسب العدد الكلي للأقلام  3*24=72، لذلك العدد الكلي للأقلام هو 72.
الرد: إنهاء[العدد الكلي للأقلام هو 72]



In [15]:
import time

start_time = time.time()
agent.run(reflexion_strategy=strategy)
end_time = time.time()

elapsed_time = end_time - start_time

print(f"Agent execution time: {elapsed_time:.2f} seconds")

Running Reflexion strategy...
سبق وأن جربت الإجابة على هذا السؤال، لكن دون نجاح. تقدم لك التقييمات التالية خطة لتجنب تكرار نفس الخطأ. استفد منها لتحسين استراتيجيتك وتقديم إجابة صحيحة على السؤال المطروح.  
Reflections:
- لقد فشلت في الإجابة على السؤال بشكل صحيح، لأنك افترضت أن الأقحوان هي الزهور الحمراء، في حين أن السؤال لم يحدد لون الزهور. لذلك، يجب عليك أن تتجنب الافتراضات غير المبررة وتركز على البيانات المعطاة في السؤال.
تفكير :
الرد: إنهاء[السؤال لم يحدد لون الزهور، لذلك لا يمكنني تحديد عدد الزهور الحمراء]
Agent execution time: 4.94 seconds


In [49]:
import re
def extract_thought(text):
  parts = text.split('الرد:')
  return parts[0]


def extract_ans(text):
  parts = text.split('\n')
  return parts[0]

def extract_ans_text(text):
    match = re.search(r"\[(.*?)\]", text)
    if match:
        return match.group(1)
    else:
        return None

t = f"""تفكير : يبدو أن هذه عبارة ترحيبية بسيطة، يمكن الرد بـ"هلا وسهلا" أو "مرحباً" كرد مناسب.
الرد: إنهاء[هلا وسهلا]
Running Reflexion strategy...
سبق وأن جربت الإجابة على هذا السؤال، لكن دون نجاح. تقدم لك التقييمات التالية خطة لتجنب تكرار نفس الخطأ. استفد منها لتحسين استراتيجيتك وتقديم إجابة صحيحة على السؤال المطروح.

Reflections:

    لقد فشلت في الإجابة على السؤال بشكل صحيح لأنك لم تتبع التعليمات بشكل دقيق. في المرة القادمة، يجب عليك قراءة التعليمات بعناية والرد بشكل مناسب ومتناسب مع المطلوب. يمكنك تجنب الخطأ السابق عن طريق التركيز على فهم السؤال والرد بشكل صحيح ودقيق. تفكير : دعونا نفكر خطوة بخطوة، السؤال هنا بسيط ولا يتطلب تفكير منطقي كبير. الرد المنطقي هو تحية بسيطة مثل "وعليكم السلام" أو "مرحباً". الرد: إنهاء[وعليكم السلام]
"""

print(extract_thought(t.split('تفكير :')[1]))
print(extract_ans_text(extract_ans(t.split('الرد:')[2]))) # 1 for 1st 2 for 2nd

 يبدو أن هذه عبارة ترحيبية بسيطة، يمكن الرد بـ"هلا وسهلا" أو "مرحباً" كرد مناسب.

وعليكم السلام


In [ ]:
الرد: مرحبا
الرد: السلام عليكم

In [ ]:
agent.run(reflexion_strategy = strategy)

In [26]:
row['answer']

'يمكن أن يكون هناك العديد من النتائج المحتملة لموقف معين حيث ترفض سامية حضور الحفل الذي تمت دعوتها إليه. بعض النتائج المحتملة تشمل:\n\n1. تفوت سامية فرصة الاختلاط وقضاء وقت ممتع مع الأصدقاء والمعارف في الحفلة.\n2. قد يشعر المضيف والضيوف الآخرون بخيبة أمل أو انزعاج بسبب عدم حضور سامية للحفلة.\n3. قد تختار سامية قضاء وقتها في المشاركة في أنشطة أخرى أو الراحة في المنزل.\n\n\nبالطبع، ستعتمد النتيجة المحددة على عوامل مختلفة، مثل طبيعة الحفلة، والعلاقة بين سامية والمضيف والضيوف الآخرين.'